In [ ]:
import numpy as np
import tensorflow as tf

import os
import json
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import cv2
from utils_train.Encoder import LabelEncoder

In [ ]:
[_tfrecords], dataset_info = tfds.load(name="coco/2017", split=["train"], with_info=True, shuffle_files=True)
for samples in _tfrecords:
    image = samples["image"]
    originalShape = tf.shape(image)[:2]
    classes = tf.cast(samples["objects"]["label"], dtype=tf.int32)
    bbox_gt = samples["objects"]["bbox"]
    break

In [ ]:
modelName = "MobileNetV3_FPN_CenterNet"

model_dir = "checkpoints/"
modelPart = modelName.split("_")

with open(os.path.join("model", "0_Config", modelName+".json"), "r") as config_file:
    config = json.load(config_file)

config['modelName'] = modelName
config['training_config']['num_classes'] = 80


##########
ids = tf.argsort(classes)
classes = tf.gather(classes, ids)
bboxes2 = tf.gather(bbox_gt, ids)
##########

le = LabelEncoder(config=config)
target = le._encode_sample(bboxes2, classes)

hm  = target[..., :80]
box  = target[..., 80:84]
rW  = target[..., 84:85]

In [ ]:
colors = np.random.rand(80, 3)*255

original_img = image.numpy()
for bbox, cls in zip(bbox_gt, classes):
    y1, x1, y2, x2 = bbox
    x1 = int(x1*image.shape[1])
    x2 = int(x2*image.shape[1])
    y1 = int(y1*image.shape[0])
    y2 = int(y2*image.shape[0])

    _text = '{}'.format(int(cls.numpy()))
    cv2.putText(original_img, _text, (x1,y1+10), cv2.FONT_HERSHEY_COMPLEX, 0.5, colors[int(cls.numpy())], thickness=1, lineType=cv2.LINE_AA)
    cv2.rectangle(original_img, (x1, y1), (x2, y2), (colors[int(cls.numpy())]), 1)

plt.figure(figsize=(12,12))
plt.imshow(cv2.resize(original_img, [480, 480]))

plt.figure(figsize=(12,12))
plt.imshow(tf.reduce_max(hm, -1).numpy(), cmap='viridis')

plt.figure(figsize=(12,12))
box_np = box.numpy()
plt.imshow(box_np)

plt.figure(figsize=(12,12))
rW_show = rW.numpy()
plt.imshow(rW_show)

a1 = tf.cast(hm==1, tf.float32)
print("Num of Positives: ", tf.reduce_sum(a1))